# Julia Bayesian Experiments
---

Import data from **HELIX project** and analyze the mixture of the chemicals 

In [1]:
using DataFrames, Statistics, Turing, LinearAlgebra, Plots, CategoricalArrays, Distributions, Parquet, StatsPlots, StatsBase
using MLDataUtils: shuffleobs, splitobs, rescale!

In [2]:
include("bayes_lib.jl")

bwqs_adv (generic function with 2 methods)

In [3]:
path = "C:\\Users\\nicol\\Documents\\Metabolomics\\data\\tables_format\\"
covariates = DataFrame(read_parquet(string(path,"covariates.parquet")));
phenotype = DataFrame(read_parquet(string(path,"phenotype.parquet")));

In [4]:
codebook = DataFrame(read_parquet(string(path,"codebook.parquet")));
show(codebook[:,[:variable_name, :domain, :family, :subfamily, :period, 
                 :var_type, :labelsshort]],
     allcols = true)

241×7 DataFrame
 Row │ variable_name           domain             family         subfamily                  period     var_type  labelsshort 
     │ String?                 String?            String?        String?                    String?    String?   String?     
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ h_abs_ratio_preg_Log    Outdoor exposures  Air Pollution  PMAbsorbance               Pregnancy  numeric   PMabs
   2 │ h_no2_ratio_preg_Log    Outdoor exposures  Air Pollution  NO2                        Pregnancy  numeric   NO2
   3 │ h_pm10_ratio_preg_None  Outdoor exposures  Air Pollution  PM10                       Pregnancy  numeric   PM10
   4 │ h_pm25_ratio_preg_None  Outdoor exposures  Air Pollution  PM2.5                      Pregnancy  numeric   PM2.5
   5 │ hs_no2_dy_hs_h_Log      Outdoor exposures  Air Pollution  NO2                        Postnatal  numeric   NO2(day)
   6 │ hs_n

In [5]:
cod1 = filter([:period, :var_type, :domain] => (x,y,z) -> x == "Pregnancy" && y == "numeric" &&
       z != "Covariates" && z!= "Phenotype",codebook);
show(cod1[:,[:variable_name, :domain, :family, :subfamily, :period, 
                 :var_type, :labelsshort]],
     allcols = true)


72×7 DataFrame
 Row │ variable_name                 domain             family             subfamily         period     var_type  labelsshort 
     │ String?                       String?            String?            String?           String?    String?   String?     
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ h_abs_ratio_preg_Log          Outdoor exposures  Air Pollution      PMAbsorbance      Pregnancy  numeric   PMabs
   2 │ h_no2_ratio_preg_Log          Outdoor exposures  Air Pollution      NO2               Pregnancy  numeric   NO2
   3 │ h_pm10_ratio_preg_None        Outdoor exposures  Air Pollution      PM10              Pregnancy  numeric   PM10
   4 │ h_pm25_ratio_preg_None        Outdoor exposures  Air Pollution      PM2.5             Pregnancy  numeric   PM2.5
   5 │ h_accesslines300_preg_dic0    Outdoor exposures  Built environment  Access            Pregnancy  numeric   BPTLine
   6 

In [6]:
phthalates = cod1[cod1.family .== "Metals",:variable_name];

In [7]:
z = covariates[completecases(covariates),:]
covart = Matrix{Float64}(z[:, phthalates]) 
target = Vector{Float64}(z[:,:e3_gac_None]);

In [8]:
for i in 1:size(phthalates)[1]
    covart[:,i] = ecdf(covart[:,i])(covart[:,i])*10
end

In [9]:
model = bwqs(covart, target)
model_adv = bwqs_adv(covart, target);

In [10]:
chain_adv = sample(model_adv, NUTS(0.65), 3000, thinning = 3)
chain = sample(model, NUTS(0.65), 3000, thinning = 3);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\nicol\.julia\packages\AdvancedHMC\51xgc\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\nicol\.julia\packages\AdvancedHMC\51xgc\src\hamiltonian.jl:47
┌ Info: Found initial step size
│   ϵ = 0.00625
└ @ Turing.Inference C:\Users\nicol\.julia\packages\Turing\Oczpc\src\inference\hmc.jl:188
Sampling: 100%|█████████████████████████████████████████| Time: 1:04:50
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, true, true, false)
└ @ AdvancedHMC C:\Users\nicol\.julia\packages\AdvancedHMC\51xgc\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, true, tr

In [11]:
hcat(quantile(chain_adv.value[:,3,1][:],[0.025,0.5,0.975]),
     quantile(chain.value[:,3,1][:],[0.025,0.5,0.975]))

3×2 Matrix{Float64}:
 -0.191766   -0.220385
 -0.123412   -0.140153
 -0.0681323  -0.0538109

In [12]:
DataFrame(Metals = phthalates,
     w = mean(Matrix(chain.value[:,4:12,1]), dims = 1)'[:],
     w_adv = mean(Matrix(chain_adv.value[:,13:21,1]), dims = 1)'[:])  


,Metals,w,w_adv
,String?,Float64,Float64
1,hs_as_m_Log2,0.0373969,0.0
2,hs_cd_m_Log2,0.185457,0.12641
3,hs_co_m_Log2,0.0348735,0.0
4,hs_cs_m_Log2,0.07788,0.0
5,hs_cu_m_Log2,0.327031,0.466527
6,hs_hg_m_Log2,0.0468121,0.030004
7,hs_mn_m_Log2,0.0615516,0.000842428
8,hs_mo_m_Log2,0.110266,0.175419
9,hs_pb_m_Log2,0.118732,0.200797
